In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn import metrics
import matplotlib.pyplot as plt
from functools import reduce
import math
from collections import Counter
import csv
import tqdm
from tqdm import tnrange, tqdm_notebook, tqdm_pandas
from datetime import datetime
%matplotlib inline

In [ ]:
def now():
    return str(datetime.now().time())[:8]
def pr(strToPrint):
    print(now() + ' '+ strToPrint)

from IPython.display import Audio
sound_file = './sound/beep2.wav'
# Used by other groups: pd.read_csv(data_path, sep="\t",encoding='utf-8',  escapechar='\\', quoting=csv.QUOTE_NONE, header=None, na_values='N')

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
columns_header = ['id', 'userId', 'createdAt', 'text', 'longitude', 'latitude', 'placeId',
                  'inReplyTo', 'source', 'truncated', 'placeLatitude', 'placeLongitude', 'sourceName', 'sourceUrl',
                 'userName', 'screenName', 'followersCount', 'friendsCount', 'statusesCount',
                 'userLocation']

# filename = os.path.join('data','sample.tsv')
filename = os.path.join('data','twex.tsv')

In [ ]:
pr('Starting to read file...')
df = pd.read_csv(filename, sep='\t', encoding='utf-8', escapechar='\\', names=columns_header,
                      quoting=csv.QUOTE_NONE, na_values='N', header=None)

pr('File is loaded!')
Audio(url=sound_file, autoplay=True)

In [ ]:
# tw = df.get_chunk()
# tw = df.head(1000000)
# tw = df2.copy()
tw = df.copy()

Making a sample of the data. You can make your own sample data with the to_pickle command.

In [ ]:
pickle_filename = os.path.join('data','head_100k_pickle.pkl')
pickle_filename2 = os.path.join('data','sample_100k_pickle.pkl')
pickle_filename3 = os.path.join('data','head_1M_pickle.pkl')
pickle_filename4 = os.path.join('data','sample_1M_pickle.pkl')
# tw.to_pickle(pickle_filename3)
tw = pd.read_pickle(pickle_filename2)

In [ ]:
tw.shape

In [ ]:
tw.head(3)

Let's count the number of "numbers" in the id section of each row to see if our data is good.

In [ ]:
def is_number_type(s):
    try:
        float(s)
    except ValueError:
        return False
    else:
        return True

In [ ]:
non_nb = 0
index_list = []
def count_nn_nb(row):
#     print(row)
    if not is_number_type(row.id):
        global non_nb
        non_nb += 1
        global index_list
        index_list.append(row.name)
pr('Starting count...')    
tw.apply(count_nn_nb, axis=1)
pr('non nb: {} / {}'.format(non_nb, len(tw)))
index_list[:10]
# is_number(tw.iloc[60].text)

In [ ]:
tw.loc[index_list]
# tw.loc[index_list[0]-1:index_list[0]+1]

<h2> Problem of  <i>\n</i>   in some of the texts</h2>

Some of the texts have the char \n and it makes a new line. This should fix the problem. However, it does not work for the whole dataset and it appears finally that this happens very rarely! So we decided to drop this.

In [ ]:
def is_nan(field):
    try:
        isNan = np.isnan(field)
    except TypeError:
        return False
    else:
        return isNan

In [ ]:
def row_is_bad(row, nextRow):
    for columnName in columns_header[4:len(columns_header)]:
        if not is_nan(row[columnName]):
            return False
    for columnName in columns_header[17:len(columns_header)]:
        if not is_nan(nextRow[columnName]):
            return False
    return True

In [ ]:
def merge_bad_rows(row, nextRow):
    new_row = {}
    for columnName in columns_header[0:3]:
        new_row[columnName] = row[columnName]
    new_row['text'] = ''.join([row.text, nextRow.id])
    for i, columnName in enumerate(columns_header[4:len(columns_header)]):
        new_row[columnName] = nextRow[columns_header[i+1]]
    return new_row

In [ ]:
rows_to_drop_list = []
rows_to_add = []
def merge_bad_lines(row, df):
    # Get next row
    index = df.index.get_loc(row.name)
    if index < len(df) - 1:
        next_row = df.iloc[index + 1]
        if row_is_bad(row, next_row):
            global rows_to_drop_list, rows_to_add            
            rows_to_add.append(merge_bad_rows(row, next_row))
            rows_to_drop_list.append(row.name)
            rows_to_drop_list.append(next_row.name)
            ## add next row to drop list    

In [ ]:
def changeBadRows(tw):
    global rows_to_drop_list, rows_to_add            
    rows_to_drop_list = []
    rows_to_add = []
    pr('Starting finding bad rows...')
    tw.apply(merge_bad_lines, args=(tw,), axis=1)
    pr('Finished finding bad rows!')
    print('Bad rows: ', rows_to_drop_list)
    print('Number of rows to add: ', len(rows_to_add))

    pr('Starting to drop bad rows...')
    tw_drop_rows = tw.drop(rows_to_drop_list)

    pr('Making new df...')
    addedRowDf = pd.DataFrame(rows_to_add, index=rows_to_drop_list[0:len(rows_to_drop_list):2])

    res = pd.concat([addedRowDf, tw_drop_rows])
    resGoodOrder = res.reindex_axis(tw.columns, axis=1)
    return resGoodOrder

Run the algorithm to change some bad rows.

In [ ]:
# tw2 = changeBadRows(tw)

Veryfing the changes:

In [ ]:
# newTw.loc[index_list[0]-2:index_list[0]+2]
# tw2.loc[index_list[2]-2:index_list[2]+2]
# tw2.loc[index_list[2]-1]

# Data cleaning - observations

Let's count the number of rows that have location data

In [ ]:
lat = 0
long = 0
def count_location_data(row):
    if not is_nan(row.latitude):
        global lat
        lat += 1 
    if not is_nan(row.longitude):
        global long
        long += 1

def execute_count_location_data(tw):
    global lat, long
    lat = 0
    long = 0
    pr('Starting computing longitude and latitude data.')
    tw.apply(count_location_data, axis=1)
    pr('Data with longitude: {:.2f}% / with latitude: {:.2f}%'.format(long/len(tw)*100, lat/len(tw)*100))

In [ ]:
execute_count_location_data(tw)

Observing the data that has a null text, and other parameters when the text is null:

In [ ]:
txtNull = tw[pd.isnull(tw['text'])]
txtNull.count()

In [ ]:
txtNull.head(2)

In [ ]:
print('There are {} / {} data with a NaN text'.format(len(txtNull), len(tw)))

In [ ]:
txtAndCreateNull = txtNull.loc[pd.isnull(txtNull['createdAt']) | pd.isnull(txtNull['text'])]
print('There are {} / {} data with a NaN text OR a NaN createdAt field.'.format(len(txtAndCreateNull), len(tw)))

In [ ]:
txtNull.loc[pd.notnull(txtNull['userId'])].head(2)

We observe that 11.641 out of a sample of 100.000 have a lot of nan values. It looks like that for 10.056 of them, all the values are nan except the id value where there is text. This could be linked to bad reading. Should be explored more. However, if we only have the text, it is not that usefull in case of event detection, so we should remove them.
Out of all of these, we have 679 with all the columns filled except the text. These should be removed too.

# Transforming the data in time series

We will drop every tweet that does not have a text and a createdAt date.

In [ ]:
tw3 = tw.dropna(axis=0, how='any', subset=['text', 'createdAt'])
print('The data have been reduced from {} tweets to {} tweets.'.format(len(tw), len(tw3)))

### Having good date format

Anomaly in a date for a specific data point

In [ ]:
# print("Lets remove the following anomaly in created date: ")
# print(df.loc[[8853944], 'createdAt'])
# df.drop(8853944, inplace=True)
# print(df.loc[[8853944], 'createdAt'])

The dates usually have the lenght of 19 chars. We have a few exceptions:

In [ ]:
pr('Exceptions:')
dateExceptions = tw3[tw3['createdAt'].str.len() != 19]
pr('We have {} exception in our data. They look like that: '.format(len(dateExceptions)))
dateExceptions.head(1)

Let's remove these exceptions:

In [ ]:
pr('Removing bad dates...')
tw4 = tw3[tw3['createdAt'].str.len() == 19]
pr('Finished.')

Let's find if other dates are not fit to be converted.

In [ ]:
pr('Starting to examine dates...')
import warnings
warnings.filterwarnings('ignore')
datetime_serie = tw4['createdAt'].convert_objects(convert_dates='coerce')
dateNotConvertible = datetime_serie[pd.isnull(datetime_serie)]
warnings.filterwarnings('default')
pr('There are {} dates that cannot be transformed.'.format(len(dateNotConvertible)))

Let's try to convert the date to datetime.

In [ ]:
pr('Starting copy...') # (to avoid transformation problems)
tw5 = tw4.copy()
pr('Converting to datetime...')
tw5['createdAt'] = pd.to_datetime(tw4['createdAt'])
pr('Setting up new indices...')
tw5.index = tw5['createdAt']
pr('Deleting old "createdAt" column...')
del tw5['createdAt']
pr('Done!')
tw5.head(2)

# Time series analysis

Let's take a look at the time series possibilities to analyze a bit the data.

In [ ]:
pr("Let's copy our data in case we change the previous part...")
ts1 = tw5.copy()
pr("Done!")

### Tweets by year

In [ ]:
fig = plt.figure()
ts1['id'].resample('A').count().plot(marker='o', color='r')
fig.suptitle('Tweets by year', fontsize=18)
plt.xlabel('Year of creation', fontsize=14)
plt.ylabel('Number of tweets', fontsize=14)

In [ ]:
fig = plt.figure()
ts1['id'].resample('M').count().plot(marker='.')
fig.suptitle('Tweets by month', fontsize=18)
plt.xlabel('Year of creation', fontsize=14)
plt.ylabel('Number of tweets (monthly)', fontsize=14)

### Tweets by month - comparison

In [ ]:
years = [2011, 2012, 2013, 2014, 2015, 2016] 
twYearly = [ts1[str(year)] for year in years]

In [ ]:
fig = plt.figure()
for i, y in enumerate(twYearly):
    shift = (len(twYearly) - 1 - i) * 12
    shitStr = str(shift) + 'M'
    y['id'].resample('M', loffset=shitStr).count().plot(label=str(years[i]), marker='.')

fig.suptitle('Tweets by month - comparison', fontsize=18)
plt.xlabel('Creation month', fontsize=14)
plt.ylabel('Number of tweets', fontsize=14)
plt.legend(loc=2)

### Most tweeted days

The days where people tweeted the most:

In [ ]:
mostTweetedDays = ts1.resample('D').count().sort_values('text', ascending=False).head(5)

In [ ]:
ts1['2012-10-30'].sample(3)

# Hashtag timeseries

To detect events, we will make a timeseries with each hashtags. We will try to detect anomalies in these time series. We will make one dictionary just with the timeseries, and another with the associated tweets.

In [ ]:
def updateDictList(dictionary, key, val):
    '''
    Updates a dictionary containing a list
    '''
    if key in dictionary:
        dictionary[key].append(val)
    else:
        dictionary[key] = [val]

In [ ]:
hts = {}
tw_h = {}
hashtag_count = 0

def makeHashtagDict(row, hts, tw_h):
    hashStart = False
    hashtagCharList = []
    text = row.text if row.text else ""
    for letter in text:
        if hashStart:
            #End of hashtag?
            if not letter.isalnum():
                if len(hashtagCharList) > 0:
                    hashtag = (''.join(hashtagCharList)).lower()
                    updateDictList(hts, hashtag, row.name)
                    updateDictList(tw_h, hashtag, row)
                hashStart = False
                hashtagCharList = []
            else:
                hashtagCharList.append(letter)
        # Start of hashtag
        if letter == '#':
            hashStart = True
            global hashtag_count
            hashtag_count += 1

In [ ]:
def executehashtagDictCreation(df):
    global hts, tw_h, hashtag_count
    hts = {}
    tw_h = {}
    hashtag_count = 0
    
    pr('Starting to create hashtag dictionaries...')
    df.apply(makeHashtagDict, args=(hts, tw_h,), axis=1)
    pr('Done!')
    print('In total, {} hashtags were detected. There are {} different ones.'.format(hashtag_count,len(hts)))

In [ ]:
executehashtagDictCreation(ts1)

## What needs to be done!
The dict contains only table => it needs to contain a time serie and not a table
The command for 1 sample could be... :

=> It needs to be done for the whole dict!

<B>HOWEVER</B> : Careful with conflict of dates!! (because it will put a "1" inthere and I don't know how it is dealt with) Which will happen and should be dealt with!

<b>Also</b>: I don't understand why there are so many conflicts of dates for some hashtags.
Is it the data that is bad, the data that is badly imported or badly dealt with?

### Number of hashtags
Let's create a dictionary that contains the counts of hashtags. And a table of it that is sorted.

In [ ]:
ht_count = {}
for key, value in hts.items():
    ht_count[key] = len(value)

In [ ]:
ht_count_sorted = sorted(ht_count, key=ht_count.get, reverse=True)

In [ ]:
i = 0
top = 15
print('Top {} hashtags: '.format(top))
while(i < top):
    ht = ht_count_sorted[i]
    i += 1
    print('{}. {} : {}'.format(i, ht, ht_count[ht]))
    [print('     ',date) for date in hts[ht][:5]]

In [ ]:
i = 0
nb_to_see = 10
for key, val in hts.items():
    if i < nb_to_see:
        i += 1
        print('{} : {}'.format(key, ht_count[key]))
        [print('    ',dd) for dd in val[:5]]
        print('-----------------------')

In [ ]:
ht_name = 'christmasrocks'
lts = pd.Series([1 for i in hts[ht_name]], index=hts[ht_name])
lts.resample('20Min').count().plot()
# lts.plot()

In [ ]:
# ts1['2016-07-18']['id'].resample('10min').count().plot()